# 智能体经典方式

目前Agent系统强烈依赖底层的大模型能力，Agent具有意图理解、任务拆解、工具调用等各种能力的同时，也继承了大模型的幻觉问题，使得Agent在处理复杂任务时可能陷入推理循环、调用错误工具等问题。为了提高Agent系统的稳定性，逐渐演变出了一些经典的系统构建方式，如以下三种：
- ReAct
- Plan-and-Solve
- Reflection

## 准备工作

开始对上述三种范式构建前，需要做一些准备工作，如环境、大模型调用密钥等，具体实现如下

In [45]:
import os
from dotenv import load_dotenv
from typing import List, Dict, Any, Callable, Optional

In [2]:
from openai import OpenAI

将模型调用相关等信息记录到.env文件中，然后运行以下代码即可加载到环境变量中

In [11]:
load_dotenv()

True

In [8]:
class ScratchAgentLLM:
    def __init__(self, model: str = None, api_key: str = None, base_url: str = None, timeout: int = None) -> None:
        self.model = model or os.getenv("LLM_MODEL_ID")
        self.api_key = api_key or os.getenv("LLM_API_KEY")
        self.base_url = base_url or os.getenv("LLM_BASE_URL")
        self.timeout = timeout or int(os.getenv("LLM_TIMEOUT", 60))

        if not all([self.model, self.api_key, self.base_url]):
            raise ValueError("Missing required environment variables")
        
        self.client = OpenAI(
            api_key=self.api_key,
            base_url=self.base_url,
            timeout = self.timeout
        )

    def think(self, messages: List[Dict[str, str]], temperature: float = 0) -> str:
        print(f"正在调用 {self.model} 模型")
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                stream=True
            )

            print(f"模型 {self.model} 调用成功")
            collected_cotent = []
            for chunk in response:
                content = chunk.choices[0].delta.content or ""
                print(content, end="", flush=True)
                collected_cotent.append(content)
            print()
            return "".join(collected_cotent)
        except Exception as e:
            print(f"模型 {self.model} 调用失败，错误信息：{e}")
            return None

使用上述定义的类调用大模型

In [9]:
client = ScratchAgentLLM()

exampleMessages = [
    {"role": "system", "content": "You are a helpful assistant that writes Python code."},
    {"role": "user", "content": "写一个快速排序算法"}
]

print("--- 调用LLM ---")
responseText = client.think(exampleMessages)
if responseText:
    print("\n\n--- 完整模型响应 ---")
    print(responseText)

--- 调用LLM ---
正在调用 kimi-latest 模型
模型 kimi-latest 调用成功
当然可以！以下是一个快速排序算法的实现代码：

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quick_sort(left) + middle + quick_sort(right)

# 示例使用
if __name__ == "__main__":
    array = [3, 6, 8, 10, 1, 2, 1]
    print("原始数组:", array)
    sorted_array = quick_sort(array)
    print("排序后的数组:", sorted_array)
```

### 代码说明：
1. **基本情况**：如果数组的长度小于或等于1，直接返回该数组，因为它已经是有序的。
2. **选择基准值**：选择数组中间的元素作为基准值（pivot）。
3. **分区操作**：
   - `left`：所有小于基准值的元素。
   - `middle`：所有等于基准值的元素。
   - `right`：所有大于基准值的元素。
4. **递归排序**：对`left`和`right`子数组递归调用`quick_sort`函数。
5. **合并结果**：将排序后的`left`、`middle`和`right`数组合并。

### 示例运行：
假设输入数组为 `[3, 6, 8, 10, 1, 2, 1]`，运行结果如下：
```
原始数组: [3, 6, 8, 10, 1, 2, 1]
排序后的数组: [1, 1, 2, 3, 6, 8, 10]
```

希望这段代码对你有帮助！


--- 完整模型响应 ---
当然可以！以

## ReAct

上述准备工作完成后，可以开始构建前文提到的范式，首先是ReAct，其核心思想是将模仿人类解决问题的方式，将推理与执行显示结合起来，形成一个思考-行动-观察的循环。思考指导行动，行动结果进一步修正思考，ReAct通过一种特殊的提示词工程来引导模型，使其每一步输出都遵循一个固定轨迹：
- 思考：分析当前情况、分解任务、执行下一步计划或反思上一步结果
- 执行：执行具体动作，通常是调用外部工具
- 观察：动作执行后返回的结果

不断重复这个Thought -> Action -> Observation的循环，将新的观察结果追加到历史记录中，形成一个不断增长的上下文，直到在思考环节中认为已经找到了最终答案，然后输出结果。这个过程形成了一个强大的协同效应：推理使得行动更具目的性，而行动则为推理提供了事实依据。

这种机制特别适用于以下场景：
- 需要外部知识的任务：如查询实时信息（天气、新闻、股价）、搜索专业领域的知识等。
- 需要精确计算的任务：将数学问题交给计算器工具，避免LLM的计算错误。
- 需要与API交互的任务：如操作数据库、调用某个服务的API来完成特定功能。

### 工具定义

工具本质上就是一个可以执行一个具体操作的函数，以搜索工具为例，其就能赋予Agent搜索的能力。如SerpApi，通过API提供结构化的Google搜索能力，能直接返“答案摘要框”或知识图谱信息

首先安装库

In [10]:
!pip install google-search-results

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32173 sha256=8ccd7348f68a2568cde7837474a593991f66e1922a67806dc02ee27d1881c3a2
  Stored in directory: c:\users\zjy\appdata\local\pip\cache\wheels\14\02\f5\4502b4e6a9e437edc881d546224eb4f068a51efd53f58f5fe7
Successfully built google-search-results


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


要使用SerpApi需要到其官方注册一个免费账号，获取密钥，然后将其保存到.env文件

一个定义良好的工具应该包含以下三个要素：
- 名称：简洁、唯一的标识符，供智能体在Action中调用，例如Search
- 描述：清晰的自然语言描述，说明工具的具体用途；其是整个机制中最关键的部分，因为大模型会依赖这段描述判断何时使用哪个工具
- 执行逻辑：真正执行任务的函数或方法

In [17]:
from serpapi import SerpApiClient

def search(query: str) -> str:
    """
    一个基于SerpApi的实战网页搜索引擎工具，其会智能地解析搜索结果，有限返回直接答案或知识图谱信息
    """
    print(f"正在执行搜索工具，查询内容：{query}")
    try:
        api_key = os.getenv("SERPAPI_API_KEY")
        if not api_key:
            return "错误：SERP_API_KEY 未设置"
        
        params = {
            "api_key": api_key,
            "engine": "google",
            "q": query,
            "gl": "cn",
            "hl": "zh-cn",
        }

        client = SerpApiClient(params)
        results = client.get_dict()

        if "answer_box_list" in results:
            return "\n".join(results["answer_box_list"])
        if "answer_box" in results and "answer" in results["answer_box"]:
            return results["answer_box"]["answer"]
        if "knowledge_graph" in results and "description" in results["knowledge_graph"]:
            return results["knowledge_graph"]["description"]
        if "organic_results" in results and results["organic_results"]:
            # 如果没有直接答案，则返回前三个有机结果的摘要
            snippets = [
                f"[{i+1}] {res.get('title', '')}\n{res.get('snippet', '')}"
                for i, res in enumerate(results["organic_results"][:3])
            ]
            return "\n\n".join(snippets)
        
        return f"对不起，没有找到关于 '{query}' 的信息。"

    except Exception as e:
        return f"搜索时发生错误: {e}"

一个Agent系统往往会需要多种工具，这样就需要一个管理器来注册和调度这些工具，故定义如下的ToolExecutor类

In [18]:
class ToolExecutor:
    def __init__(self) -> None:
        self.tool: Dict[str, Dict[str, Any]] = {}
    
    def register_tool(self, name: str, description: str, func: Callable) -> None:
        if name in self.tool:
            print(f"工具 '{name}' 已存在，将被覆盖")
        self.tool[name] = {
            "description": description,
            "func": func
        }
        print(f"工具 '{name}' 已注册")
    
    def get_tool(self, name: str) -> Callable:
        return self.tool.get(name, {}).get("func")
    
    def get_available_tools(self) -> str:
        return "\n".join([f"{name}: {tool['description']}" for name, tool in self.tool.items()])

将search工具注册到ToolExecutor中，并模拟一次调用，以验证整个流程是否正常工作

In [19]:
# 1. 初始化工具执行器
tool_executor = ToolExecutor()

# 2. 注册我们的实战搜索工具
search_description = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
tool_executor.register_tool("Search", search_description, search)

# 3. 打印可用的工具
print("\n--- 可用的工具 ---")
print(tool_executor.get_available_tools())

# 4. 智能体的Action调用，这次我们问一个实时性的问题
print("\n--- 执行 Action: Search['英伟达最新的GPU型号是什么'] ---")
tool_name = "Search"
tool_input = "英伟达最新的GPU型号是什么"

tool_function = tool_executor.get_tool(tool_name)
if tool_function:
    observation = tool_function(tool_input)
    print("--- 观察 (Observation) ---")
    print(observation)
else:
    print(f"错误:未找到名为 '{tool_name}' 的工具。")

工具 'Search' 已注册

--- 可用的工具 ---
Search: 一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。

--- 执行 Action: Search['英伟达最新的GPU型号是什么'] ---
正在执行搜索工具，查询内容：英伟达最新的GPU型号是什么
--- 观察 (Observation) ---
[1] 比较GeForce 系列最新一代显卡和前代显卡
比较最新一代RTX 30 系列显卡和前代的RTX 20 系列、GTX 10 和900 系列显卡。查看规格、功能、技术支持等内容。

[2] GeForce 显卡| NVIDIA
显卡 · GeForce RTX 50 系列 · GeForce RTX 40 系列.

[3] GeForce RTX 50 系列显卡
GeForce RTX™ 50 系列GPU 搭载NVIDIA Blackwell 架构，为游戏玩家和创作者带来全新玩法。RTX 50 系列具备强大的AI 算力，带来升级体验和更逼真的画面。


### ReAct编码实现

- 系统提示词设计：提示词是整个ReAct机制的基石，为大模型提供行行动的操作指令；需要一个精心设计的模板，将可用工具、用户问题以及中间步骤交互、有机结合为历史信息

In [20]:
# ReAct 提示词模板
REACT_PROMPT_TEMPLATE = """
请注意，你是一个有能力调用外部工具的智能助手。

可用工具如下:
{tools}

请严格按照以下格式进行回应:

Thought: 你的思考过程，用于分析问题、拆解任务和规划下一步行动。
Action: 你决定采取的行动，必须是以下格式之一:
- `{{tool_name}}[{{tool_input}}]`:调用一个可用工具。
- `Finish[最终答案]`:当你认为已经获得最终答案时。
- 当你收集到足够的信息，能够回答用户的最终问题时，你必须在Action:字段后使用 finish(answer="...") 来输出最终答案。

现在，请开始解决以下问题:
Question: {question}
History: {history}
"""


这个模板定义智能体与LLM之间交互的规范：
- 角色定义：定义大模型的角色
- 工具清单：告诉大模型有哪些可用的“手脚”，可以与外界环境交流
- 格式约束：最重要的部分，强制大模型输出具有结构性，使得能够通过代码精确解析其意图
- 动态上下文：将用户的原始问题和不断积累的交互历史注入，让大模型基于完整的上下文进行决策

- 核心循环实现：ReActAgent的核心是一个循环，不断地“格式化提示词->调用LLM->整合结果”，直到任务完成或达到最大步数限制

In [21]:
import re

In [32]:
class ReActAgent:
    def __init__(self, llm: ScratchAgentLLM, tool_executor: ToolExecutor, max_steps: int = 5) -> None:
        self.llm = llm
        self.tool_executor = tool_executor
        self.max_steps = max_steps
        self.history = []
    
    def run(self, question: str) -> str:
        """
        运行ReActAgent，解决用户的问题
        """
        self.history = []
        current_step = 0

        while current_step < self.max_steps:
            current_step += 1
            print(f"\n--- 第 {current_step} 步 ---")

            # 1. 格式化提示词
            prompt = REACT_PROMPT_TEMPLATE.format(
                tools=self.tool_executor.get_available_tools(),
                question=question,
                history="\n".join(self.history)
            )
            print("--- 提示词 ---")
            print(prompt)

            # 2. 调用LLM
            messages = [{"role": "user", "content": prompt}]
            response = self.llm.think(messages)

            if not response:
                print("错误:LLM未能返回有效响应。")
                break

            # 3. 解析LLM的输出
            thought, action = self._parse_output(response)

            if thought:
                print(f"思考: {thought}")
            
            if not action:
                print("错误:未能从LLM的输出中提取到有效的Action。")
                continue
            
            # 4. 执行Action
            if action.startswith("finish"):
                final_answer = self._parse_action_input(action)
                print(f"🎉 最终答案: {final_answer}")
                return final_answer
            
            tool_name, tool_input = self._parse_action(action)
            if not tool_name or not tool_input:
                continue
            
            # 5. 调用工具
            print(f"🔍 调用工具: {tool_name}[{tool_input}]")
            tool_func = self.tool_executor.get_tool(tool_name)
            if not tool_func:
                observation = f"错误:未能找到名为 '{tool_name}' 的工具。"
            else:
                try:
                    observation = tool_func(tool_input)
                except Exception as e:
                    observation = f"错误:工具 '{tool_name}' 执行失败，错误信息: {e}"
            print(f"观察: {observation}")

            # 6. 更新历史
            self.history.append(f"Action: {action}")
            self.history.append(f"Observation: {observation}")
        
        print("🚫 达到最大步数限制，无法继续执行。")
        return None
    
    def _parse_output(self, text: str):
        """解析LLM的输出，提取Thought和Action。"""
        thought_match = re.search(r"Thought: (.*)", text)
        action_match = re.search(r"Action: (.*)", text)
        thought = thought_match.group(1).strip() if thought_match else None
        action = action_match.group(1).strip() if action_match else None
        return thought, action

    def _parse_action(self, action_text: str):
        """解析Action字符串，提取工具名称和输入。"""
        match = re.match(r"(\w+)\[(.*)\]", action_text)
        if match:
            return match.group(1), match.group(2)
        return None, None
    
    def _parse_action_input(self, action_text: str):
        """
        更加健壮地解析Action字符串，提取输入内容，支持[]和()两种括号，考虑异常情况。
        如: finish[最终答案内容] 或 finish(最终答案内容)
        """
        if not isinstance(action_text, str):
            return ""
        # 支持 finish[xxx] 或 finish(xxx)
        for pattern in [r"\w+\[(.*)\]", r"\w+\((.*)\)"]:
            match = re.match(pattern, action_text)
            if match:
                return match.group(1).strip()
        return ""


In [33]:
llm = ScratchAgentLLM()
tool_executor = ToolExecutor()
search_desc = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
tool_executor.register_tool("Search", search_desc, search)
agent = ReActAgent(llm=llm, tool_executor=tool_executor)
question = "一加Ace 6和一加Ace 6 T两款手机有什么区别？"
agent.run(question)

工具 'Search' 已注册

--- 第 1 步 ---
--- 提示词 ---

请注意，你是一个有能力调用外部工具的智能助手。

可用工具如下:
Search: 一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。

请严格按照以下格式进行回应:

Thought: 你的思考过程，用于分析问题、拆解任务和规划下一步行动。
Action: 你决定采取的行动，必须是以下格式之一:
- `{tool_name}[{tool_input}]`:调用一个可用工具。
- `Finish[最终答案]`:当你认为已经获得最终答案时。
- 当你收集到足够的信息，能够回答用户的最终问题时，你必须在Action:字段后使用 finish(answer="...") 来输出最终答案。

现在，请开始解决以下问题:
Question: 一加Ace 6和一加Ace 6 T两款手机有什么区别？
History: 

正在调用 kimi-latest 模型
模型 kimi-latest 调用成功
Thought: 为了回答关于一加Ace 6和一加Ace 6 T两款手机的区别，我需要查找这两款手机的详细规格和特性。这可以通过搜索相关的手机评测、官方文档或技术规格来完成。

Action: Search["一加Ace 6 和 一加Ace 6 T 区别"]
思考: 为了回答关于一加Ace 6和一加Ace 6 T两款手机的区别，我需要查找这两款手机的详细规格和特性。这可以通过搜索相关的手机评测、官方文档或技术规格来完成。
🔍 调用工具: Search["一加Ace 6 和 一加Ace 6 T 区别"]
正在执行搜索工具，查询内容："一加Ace 6 和 一加Ace 6 T 区别"
观察: [1] 一加Ace 6T和一加Ace 6怎么选？关键区别总结，看完就不纠结
一加Ace 6T 起售价2159 元，一加Ace 6 起售价2339 元。 在价格更低的情况下，Ace 6T 还给你更稳的骁龙8 Gen5 和更大的8300mAh 电池。 要 ...

[2] 一加Ace6T上市预约，对比旧款一加Ace6有什么升级
朋友们，您们好，这里是老翟谈数码，一加Ace6T已经开启了预约，那么对比之前旧款一加Ace6有什么升级，选哪个好？如下详细盘点这两款手机的区别对比和选购建

''

## Plan-and-Solve

本质就是先规划，后执行：
- 规划阶段：接收问题后首先将问题分解，指定一个清晰、可执行的分步计划，这是依次大模型调用
- 指定阶段：严格按照计划逐一执行；每一步执行可能是依次独立的大模型调用，或者是对上一步结果的加工，直到嘉华中的所有步骤完成

为了凸显Plan-and-Solve范式在结构化推理任务上的优势，将不使用工具的方式，而是通过提示词的设计，完成一个推理任务。这类任务的特点是，答案无法通过单次查询或计算得出，必须先将问题分解为一系列逻辑连贯的子步骤，然后按顺序求解。这恰好能发挥Plan-and-Solve“先规划，后执行”的核心能力。

以目标问题是：“一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？”为例

规划阶段的目标是让大语言模型接收原始问题，并输出一个清晰、分步骤的行动计划。这个计划必须是结构化的，以便代码可以轻松解析并逐一执行。因此设计的提示词需要明确地告诉模型它的角色和任务，并给出一个输出格式的范例。

In [34]:
PLANNER_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。你的任务是将用户提出的复杂问题分解成一个由多个简单步骤组成的行动计划。
请确保计划中的每个步骤都是一个独立的、可执行的子任务，并且严格按照逻辑顺序排列。
你的输出必须是一个Python列表，其中每个元素都是一个描述子任务的字符串。

问题: {question}

请严格按照以下格式输出你的计划,```python与```作为前后缀是必要的:
```python
["步骤1", "步骤2", "步骤3", ...]
```
"""


这个提示词通过以下几点确保了输出的质量和稳定性：

- 角色设定： “顶级的AI规划专家”，激发模型的专业能力。
- 任务描述： 清晰地定义了“分解问题”的目标。
- 格式约束： 强制要求输出为一个Python列表格式的字符串，这极大地简化了后续代码的解析工作，使其比解析自然语言更稳定、更可靠。

In [38]:
class Planner:
    def __init__(self, llm: ScratchAgentLLM):
        self.llm = llm

    def plan(self, question: str) -> list[str]:
        """
        根据用户问题生成一个行动计划。
        """
        prompt = PLANNER_PROMPT_TEMPLATE.format(question=question)
        
        # 为了生成计划，我们构建一个简单的消息列表
        messages = [{"role": "user", "content": prompt}]
        
        print("--- 正在生成计划 ---")
        # 使用流式输出来获取完整的计划
        response_text = self.llm.think(messages=messages) or ""
        
        print(f"✅ 计划已生成:\n{response_text}")
        
        # 解析LLM输出的列表字符串
        try:
            # 找到```python和```之间的内容
            plan_str = response_text.split("```python")[1].split("```")[0].strip()
            # 使用ast.literal_eval来安全地执行字符串，将其转换为Python列表
            plan = eval(plan_str)
            return plan if isinstance(plan, list) else []
        except (ValueError, SyntaxError, IndexError) as e:
            print(f"❌ 解析计划时出错: {e}")
            print(f"原始响应: {response_text}")
            return []
        except Exception as e:
            print(f"❌ 解析计划时发生未知错误: {e}")
            return []


在规划器(Planner)生成了清晰的行动蓝图后，就需要一个执行器(Executor)来逐一完成计划中的任务。执行器不仅负责调用大模型来解决每个子问题，还承担着一个至关重要的角色：状态管理。它必须记录每一步的执行结果，并将其作为上下文提供给后续步骤，确保信息在整个任务链条中顺畅流动

执行器的提示词与规划器不同。它的目标不是分解问题，而是在已有上下文的基础上，专注解决当前这一个步骤。因此，提示词需要包含以下关键信息：

- 原始问题： 确保模型始终了解最终目标。
- 完整计划： 让模型了解当前步骤在整个任务中的位置。
- 历史步骤与结果： 提供至今为止已经完成的工作，作为当前步骤的直接输入。
- 当前步骤： 明确指示模型现在需要解决哪一个具体任务。

In [35]:
EXECUTOR_PROMPT_TEMPLATE = """
你是一位顶级的AI执行专家。你的任务是严格按照给定的计划，一步步地解决问题。
你将收到原始问题、完整的计划、以及到目前为止已经完成的步骤和结果。
请你专注于解决“当前步骤”，并仅输出该步骤的最终答案，不要输出任何额外的解释或对话。

# 原始问题:
{question}

# 完整计划:
{plan}

# 历史步骤与结果:
{history}

# 当前步骤:
{current_step}

请仅输出针对“当前步骤”的回答:
"""


将执行逻辑封装到Executor类中。这个类将循环遍历计划，调用LLM，并维护一个历史记录（状态）

In [42]:
class Executor:
    def __init__(self, llm: ScratchAgentLLM):
        self.llm = llm

    def execute(self, question: str, plan: list[str]) -> str:
        """
        根据计划，逐步执行并解决问题。
        """
        history = "" # 用于存储历史步骤和结果的字符串
        
        print("\n--- 正在执行计划 ---")
        
        for i, step in enumerate(plan):
            print(f"\n-> 正在执行步骤 {i+1}/{len(plan)}: {step}")
            
            prompt = EXECUTOR_PROMPT_TEMPLATE.format(
                question=question,
                plan=plan,
                history=history if history else "无", # 如果是第一步，则历史为空
                current_step=step
            )
            print("\n--- 提示词 ---")
            print(prompt)
            
            messages = [{"role": "user", "content": prompt}]
            
            response_text = self.llm.think(messages=messages) or ""
            
            # 更新历史记录，为下一步做准备
            history += f"步骤 {i+1}: {step}\n结果: {response_text}\n\n"
            
            print(f"✅ 步骤 {i+1} 已完成，结果: {response_text}")

        # 循环结束后，最后一步的响应就是最终答案
        final_answer = response_text
        return final_answer
   

现在已经分别构建了负责“规划”的Planner和负责“执行”的Executor。最后一步是将这两个组件整合到一个统一的智能体PlanAndSolveAgent中，并赋予它解决问题的完整能力。将创建一个主类PlanAndSolveAgent，它的职责非常清晰：接收一个LLM客户端，初始化内部的规划器和执行器，并提供一个简单的run方法来启动整个流程。

In [43]:
class PlanAndSolveAgent:
    def __init__(self, llm: ScratchAgentLLM):
        """
        初始化智能体，同时创建规划器和执行器实例。
        """
        self.llm = llm
        self.planner = Planner(self.llm)
        self.executor = Executor(self.llm)

    def run(self, question: str):
        """
        运行智能体的完整流程:先规划，后执行。
        """
        print(f"\n--- 开始处理问题 ---\n问题: {question}")
        
        # 1. 调用规划器生成计划
        plan = self.planner.plan(question)
        
        # 检查计划是否成功生成
        if not plan:
            print("\n--- 任务终止 --- \n无法生成有效的行动计划。")
            return

        # 2. 调用执行器执行计划
        final_answer = self.executor.execute(question, plan)
        
        print(f"\n--- 任务完成 ---\n最终答案: {final_answer}")


In [44]:
llm = ScratchAgentLLM()
agent = PlanAndSolveAgent(llm)
question = "一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？"
agent.run(question)


--- 开始处理问题 ---
问题: 一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？
--- 正在生成计划 ---
正在调用 kimi-latest 模型
模型 kimi-latest 调用成功
```python
["确定周一卖出的苹果数量为15个",
 "计算周二卖出的苹果数量，即周一数量的两倍",
 "计算周三卖出的苹果数量，即周二数量减去5个",
 "将周一、周二和周三卖出的苹果数量相加，得到总数量"]
```
✅ 计划已生成:
```python
["确定周一卖出的苹果数量为15个",
 "计算周二卖出的苹果数量，即周一数量的两倍",
 "计算周三卖出的苹果数量，即周二数量减去5个",
 "将周一、周二和周三卖出的苹果数量相加，得到总数量"]
```

--- 正在执行计划 ---

-> 正在执行步骤 1/4: 确定周一卖出的苹果数量为15个

--- 提示词 ---

你是一位顶级的AI执行专家。你的任务是严格按照给定的计划，一步步地解决问题。
你将收到原始问题、完整的计划、以及到目前为止已经完成的步骤和结果。
请你专注于解决“当前步骤”，并仅输出该步骤的最终答案，不要输出任何额外的解释或对话。

# 原始问题:
一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？

# 完整计划:
['确定周一卖出的苹果数量为15个', '计算周二卖出的苹果数量，即周一数量的两倍', '计算周三卖出的苹果数量，即周二数量减去5个', '将周一、周二和周三卖出的苹果数量相加，得到总数量']

# 历史步骤与结果:
无

# 当前步骤:
确定周一卖出的苹果数量为15个

请仅输出针对“当前步骤”的回答:

正在调用 kimi-latest 模型
模型 kimi-latest 调用成功
15
✅ 步骤 1 已完成，结果: 15

-> 正在执行步骤 2/4: 计算周二卖出的苹果数量，即周一数量的两倍

--- 提示词 ---

你是一位顶级的AI执行专家。你的任务是严格按照给定的计划，一步步地解决问题。
你将收到原始问题、完整的计划、以及到目前为止已经完成的步骤和结果。
请你专注于解决“当前步骤”，并仅输出该步骤的最终

## Reflection

已经实现的ReAct和Plan-and-Solve范式中，Agent一旦完成了任务，其工作流程便告结束。然而，它们生成的初始答案，无论是行动轨迹还是最终结果，都可能存在谬误或有待改进之处。Reflection机制的核心思想，正是为Agent引入一种事后的自我校正循环，使其能够像人类一样，审视自己的工作，发现不足，并进行迭代优化。主要环节如下：

- 执行：使用上述前两项架构的Agent系统生成一个初稿
- 反思：调用一个独立的、或带有特殊提示词的模型，扮演评审员，对初稿进行多维度评估，得到反馈
  - 事实性错误
  - 逻辑漏洞
  - 效率问题
  - 遗漏信息
- 优化：将初稿和反馈作为新的上下文，再次调用大模型得到最终的修订稿

为了体现Reflection机制，需要引入记忆管理机制，因为reflection通常对应着信息的存储和提取，如果上下文足够长时，让评审模型直接获取所有的信息然后进行反思往往会传入很多冗余信息。以“编写一个Python函数，找出1到n之间所有的素数 (prime numbers)。”为例，这个任务是检验Reflection机制的绝佳场景：

- 存在明确的优化路径：大语言模型初次生成的代码很可能是一个简单但效率低下的递归实现。
- 反思点清晰：可以通过反思发现其“时间复杂度过高”或“存在重复计算”的问题。
- 优化方向明确：可以根据反馈，将其优化为更高效的迭代版本或使用备忘录模式的版本。

In [48]:
class Memory:
    """
    一个简单的短期记忆模块，用于存储智能体的行动与反思轨迹。
    """

    def __init__(self):
        """
        初始化一个空列表来存储所有记录。
        """
        self.records: List[Dict[str, Any]] = []

    def add_record(self, record_type: str, content: str):
        """
        向记忆中添加一条新记录。

        参数:
        - record_type (str): 记录的类型 ('execution' 或 'reflection')。
        - content (str): 记录的具体内容 (例如，生成的代码或反思的反馈)。
        """
        record = {"type": record_type, "content": content}
        self.records.append(record)
        print(f"📝 记忆已更新，新增一条 '{record_type}' 记录。")

    def get_trajectory(self) -> str:
        """
        将所有记忆记录格式化为一个连贯的字符串文本，用于构建提示词。
        """
        trajectory_parts = []
        for record in self.records:
            if record['type'] == 'execution':
                trajectory_parts.append(f"--- 上一轮尝试 (代码) ---\n{record['content']}")
            elif record['type'] == 'reflection':
                trajectory_parts.append(f"--- 评审员反馈 ---\n{record['content']}")
        
        return "\n\n".join(trajectory_parts)

    def get_last_execution(self) -> Optional[str]:
        """
        获取最近一次的执行结果 (例如，最新生成的代码)。
        如果不存在，则返回 None。
        """
        for record in reversed(self.records):
            if record['type'] == 'execution':
                return record['content']
        return None


有了Memory模块作为基础，可以着手构建ReflectionAgent的核心逻辑。整个智能体的工作流程将围绕之前讨论的“执行-反思-优化”循环展开，并通过精心设计的提示词来引导大语言模型扮演不同的角色。

初始执行提示词(Execution Prompt)：这是智能体首次尝试解决问题的提示词，内容相对直接，只要求模型完成指定任务。

In [46]:
INITIAL_PROMPT_TEMPLATE = """
你是一位资深的Python程序员。请根据以下要求，编写一个Python函数。
你的代码必须包含完整的函数签名、文档字符串，并遵循PEP 8编码规范。

要求: {task}

请直接输出代码，不要包含任何额外的解释。
"""


反思提示词(Reflection Prompt)：这个提示词是Reflection机制的灵魂。它指示模型扮演“代码评审员”的角色，对上一轮生成的代码进行批判性分析，并提供具体的、可操作的反馈。

In [49]:
REFLECT_PROMPT_TEMPLATE = """
你是一位极其严格的代码评审专家和资深算法工程师，对代码的性能有极致的要求。
你的任务是审查以下Python代码，并专注于找出其在<strong>算法效率</strong>上的主要瓶颈。

# 原始任务:
{task}

# 待审查的代码:
```python
{code}
```

请分析该代码的时间复杂度，并思考是否存在一种<strong>算法上更优</strong>的解决案方来显著提升性能。
如果存在，请清晰地指出当前算法的不足，并提出具体的、可行的改进算法建议（例如，使用筛法替代试除法）。
如果代码在算法层面已经达到最优，才能回答“无需改进”。

请直接输出你的反馈，不要包含任何额外的解释。
"""


优化提示词(Refinement Prompt)：当收到反馈后，这个提示词将引导模型根据反馈内容，对原有代码进行修正和优化。

In [50]:

REFINE_PROMPT_TEMPLATE = """
你是一位资深的Python程序员。你正在根据一位代码评审专家的反馈来优化你的代码。

# 原始任务:
{task}

# 你上一轮尝试的代码:
{last_code_attempt}
评审员的反馈：
{feedback}

请根据评审员的反馈，生成一个优化后的新版本代码。
你的代码必须包含完整的函数签名、文档字符串，并遵循PEP 8编码规范。
请直接输出优化后的代码，不要包含任何额外的解释。
"""


将上述实现的Memory模块和一套提示词封装为ReflectionAgent类

In [51]:
class ReflectionAgent:
    def __init__(self, llm: ScratchAgentLLM, max_iterations=3):
        self.llm = llm
        self.memory = Memory()
        self.max_iterations = max_iterations

    def run(self, task: str):
        print(f"\n--- 开始处理任务 ---\n任务: {task}")

        # --- 1. 初始执行 ---
        print("\n--- 正在进行初始尝试 ---")
        initial_prompt = INITIAL_PROMPT_TEMPLATE.format(task=task)
        initial_code = self._get_llm_response(initial_prompt)
        self.memory.add_record("execution", initial_code)

        # --- 2. 迭代循环:反思与优化 ---
        for i in range(self.max_iterations):
            print(f"\n--- 第 {i+1}/{self.max_iterations} 轮迭代 ---")

            # a. 反思
            print("\n-> 正在进行反思...")
            last_code = self.memory.get_last_execution()
            reflect_prompt = REFLECT_PROMPT_TEMPLATE.format(task=task, code=last_code)
            feedback = self._get_llm_response(reflect_prompt)
            self.memory.add_record("reflection", feedback)

            # b. 检查是否需要停止
            if "无需改进" in feedback:
                print("\n✅ 反思认为代码已无需改进，任务完成。")
                break

            # c. 优化
            print("\n-> 正在进行优化...")
            refine_prompt = REFINE_PROMPT_TEMPLATE.format(
                task=task,
                last_code_attempt=last_code,
                feedback=feedback
            )
            refined_code = self._get_llm_response(refine_prompt)
            self.memory.add_record("execution", refined_code)
        
        final_code = self.memory.get_last_execution()
        print(f"\n--- 任务完成 ---\n最终生成的代码:\n```python\n{final_code}\n```")
        return final_code

    def _get_llm_response(self, prompt: str) -> str:
        """一个辅助方法，用于调用LLM并获取完整的流式响应。"""
        messages = [{"role": "user", "content": prompt}]
        response_text = self.llm.think(messages=messages) or ""
        return response_text


In [52]:
llm = ScratchAgentLLM()
agent = ReflectionAgent(llm)
task = "编写一个Python函数，找出1到n之间所有的素数 (prime numbers)。"
agent.run(task)


--- 开始处理任务 ---
任务: 编写一个Python函数，找出1到n之间所有的素数 (prime numbers)。

--- 正在进行初始尝试 ---
正在调用 kimi-latest 模型
模型 kimi-latest 调用成功
```python
def find_primes(n):
    """
    Finds all prime numbers between 1 and n.

    Parameters:
    n (int): The upper limit to find prime numbers.

    Returns:
    list: A list of prime numbers between 1 and n.
    """
    if n < 2:
        return []
    
    is_prime = [True] * (n + 1)
    is_prime[0] = is_prime[1] = False
    
    for i in range(2, int(n**0.5) + 1):
        if is_prime[i]:
            for j in range(i*i, n + 1, i):
                is_prime[j] = False
    
    return [i for i in range(2, n + 1) if is_prime[i]]
```
📝 记忆已更新，新增一条 'execution' 记录。

--- 第 1/3 轮迭代 ---

-> 正在进行反思...
正在调用 kimi-latest 模型
模型 kimi-latest 调用成功
该代码使用了经典的埃拉托斯特尼筛法（Sieve of Eratosthenes），其时间复杂度为 \(O(n \log \log n)\)，空间复杂度为 \(O(n)\)。这种算法在寻找1到n之间的所有素数时已经非常高效，且在实际应用中表现良好。

然而，如果要进一步优化，可以考虑以下几点：
1. **减少空间复杂度**：可以使用位数组来存储素数状态，从而将空间复杂度从 \(O(n)\) 降低到 \(O(n/8)\)。
2. **减少不必要的迭代**：在标记非素

'```python\ndef find_primes(n):\n    """\n    Finds all prime numbers between 1 and n using an optimized Sieve of Eratosthenes algorithm.\n\n    Parameters:\n    n (int): The upper limit to find prime numbers.\n\n    Returns:\n    list: A list of prime numbers between 1 and n.\n    """\n    if n < 2:\n        return []\n    \n    # Use a boolean array to reduce space complexity\n    is_prime = [True] * (n + 1)\n    is_prime[0] = is_prime[1] = False  # 0 and 1 are not prime numbers\n    \n    for i in range(2, int(n**0.5) + 1):\n        if is_prime[i]:\n            for j in range(i * i, n + 1, i):\n                is_prime[j] = False\n    \n    return [i for i in range(2, n + 1) if is_prime[i]]\n```'